In [ ]:
import os
os.chdir("C:/Users/yotam/code_projects/APDTFlow")
print("New working directory:", os.getcwd())

New working directory: C:\Users\yotam\code_projects\APDTFlow


In [ ]:
import sys
import os

# If the notebook is in "experiments/notebooks/", then the project root is two levels up.
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)


Project root added to sys.path: C:\Users\yotam


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

CUDA available: False


In [ ]:
from torch.utils.data import DataLoader
from apdtflow.data import TimeSeriesWindowDataset

# Define dataset parameters
csv_file = "C:/Users/yotam/code_projects/APDTFlow/dataset_examples/Electric_Production.csv"
date_col = "DATE"
value_col = "IPG2211A2N"
T_in = 12    # Number of past time steps (input sequence length)
T_out = 3    # Forecast horizon (number of future time steps)

# Create dataset and DataLoader
dataset = TimeSeriesWindowDataset(csv_file=csv_file,
                                  date_col=date_col,
                                  value_col=value_col,
                                  T_in=T_in,
                                  T_out=T_out)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

print("Dataset loaded. Total samples:", len(dataset))

## Model Initialization and Training

In [ ]:
from apdtflow.models.apdtflow import APDTFlow
from apdtflow.models import transformer_forecaster, tcn_forecaster, ensemble_forecaster

# Initialize models with common parameters
num_scales = 3
input_channels = 1
filter_size = 5
hidden_dim = 16
output_dim = 1
forecast_horizon = T_out  # Should match T_out

# APDTFlow model
apdtflow_model = APDTFlow(num_scales=num_scales,
                          input_channels=input_channels,
                          filter_size=filter_size,
                          hidden_dim=hidden_dim,
                          output_dim=output_dim,
                          forecast_horizon=forecast_horizon)
apdtflow_model.to(device)

### alternative forecaster models

In [ ]:
try:
    from apdtflow.models.transformer_forecaster import TransformerForecaster
    from apdtflow.models.tcn_forecaster import TCNForecaster
    from apdtflow.models.ensemble_forecaster import EnsembleForecaster
except ImportError:
    print("One or more alternative forecaster modules are not available.")

# Initialize alternative models (if available)
transformer_model = TransformerForecaster(num_scales=num_scales,
                                          input_channels=input_channels,
                                          filter_size=filter_size,
                                          hidden_dim=hidden_dim,
                                          output_dim=output_dim,
                                          forecast_horizon=forecast_horizon)
transformer_model.to(device)

tcn_model = TCNForecaster(num_scales=num_scales,
                          input_channels=input_channels,
                          filter_size=filter_size,
                          hidden_dim=hidden_dim,
                          output_dim=output_dim,
                          forecast_horizon=forecast_horizon)
tcn_model.to(device)

ensemble_model = EnsembleForecaster(num_scales=num_scales,
                                    input_channels=input_channels,
                                    filter_size=filter_size,
                                    hidden_dim=hidden_dim,
                                    output_dim=output_dim,
                                    forecast_horizon=forecast_horizon)
ensemble_model.to(device)

## Training Function

In [ ]:
def train_forecaster(model, train_loader, num_epochs, learning_rate, device):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            if x_batch.dim() == 4 and x_batch.size(1) == 1:
                x_batch = x_batch.squeeze(1)
            batch_size, _, T_in_current = x_batch.size()
            t_span = torch.linspace(0, 1, steps=T_in_current, device=device)
            
            optimizer.zero_grad()
            preds, pred_logvars = model(x_batch, t_span)
            mse = (preds - y_batch.transpose(1, 2)) ** 2
            loss = torch.mean(0.5 * (mse / (pred_logvars.exp() + 1e-6)) + 0.5 * pred_logvars)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * batch_size
        avg_loss = epoch_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
    return model


## Train Each Model

In [ ]:
num_epochs = 5 
learning_rate = 0.001

print("Training APDTFlow model...")
trained_apdtflow = train_forecaster(apdtflow_model, train_loader, num_epochs, learning_rate, device)

print("\nTraining TransformerForecaster model...")
trained_transformer = train_forecaster(transformer_model, train_loader, num_epochs, learning_rate, device)

print("\nTraining TCNForecaster model...")
trained_tcn = train_forecaster(tcn_model, train_loader, num_epochs, learning_rate, device)

print("\nTraining EnsembleForecaster model...")
trained_ensemble = train_forecaster(ensemble_model, train_loader, num_epochs, learning_rate, device)

## Evaluation

In [ ]:
print("\nEvaluating APDTFlow model...")
mse, mae = trained_apdtflow.evaluate(train_loader, device)
print(f"APDTFlow Evaluation -> MSE: {mse:.4f}, MAE: {mae:.4f}")


## Similarly, evaluate the alternative models

In [ ]:
mse_trans, mae_trans = trained_transformer.evaluate(train_loader, device)
print(f"TransformerForecaster Evaluation -> MSE: {mse_trans:.4f}, MAE: {mae_trans:.4f}")

mse_tcn, mae_tcn = trained_tcn.evaluate(train_loader, device)
print(f"TCNForecaster Evaluation -> MSE: {mse_tcn:.4f}, MAE: {mae_tcn:.4f}")

mse_ensemble, mae_ensemble = trained_ensemble.evaluate(train_loader, device)
print(f"EnsembleForecaster Evaluation -> MSE: {mse_ensemble:.4f}, MAE: {mae_ensemble:.4f}")